# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2
#### and make it python 2 and 3 compatible by Hao Xi.

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015, named as citibike)

In [1]:
SQL_SOURCE = 'https://hx517.carto.com/api/v2/sql?q='

try:
    from urllib.request import urlopen
except ImportError:
    from urllib2 import urlopen
try:
    from urllib.error import HTTPError
except ImportError:
    from urllib2 import HTTPError
try:
    from urllib.parse import urlencode
except ImportError:
    from urllib import urlencode

import urllib
import urllib2
import io
import ast
import pandas as pd
import StringIO
# try to make it python 2 and 3 compatible.

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-77bb9da7299a>, line 38)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

## Task 1¶
### a. Sort data by start_station_id - only checking trips with duration <= 3 hours
#### 1. the top 10 records

In [ ]:
test = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY start_station_id
LIMIT 10
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')

#### 2. the last 10 records

In [ ]:
test = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY start_station_id DESC
LIMIT 10
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

### b. Sort data by tripduration - only checking trips with duration <= 3 hours
#### 1. the top 10 records

In [ ]:
test = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY tripduration
LIMIT 10
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

#### 2. the last 10 records

In [ ]:
test = '''
SELECT * FROM citibike
WHERE tripduration<10800
ORDER BY tripduration DESC
LIMIT 10
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

### c. List all unique start_station_id values

In [ ]:
test = '''
SELECT DISTINCT start_station_id FROM citibike
ORDER BY start_station_id
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

### d. Count the number/average/min/max of trips

In [ ]:
test = '''
SELECT COUNT(tripduration) as numtrips,
       AVG(tripduration) as avgtrips,
       MIN(tripduration) as mintrips,
       MAX(tripduration) as maxtrips
FROM citibike
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

## Task 2
### a. Selecting trips started on Feb-02-2015 only

In [ ]:
test = '''
SELECT * FROM citibike
WHERE starttime >= '2015-02-02 00:00'
AND   starttime  < '2015-02-03 00:00'
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

### b. Selecting trips started on the weekends and calculate the average trip duration during weekends

In [ ]:
test = '''
SELECT * FROM citibike
WHERE extract(DOW FROM starttime) in (0,6)
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

In [ ]:
test = '''
SELECT AVG(tripduration) as avgtrips FROM citibike
WHERE extract(DOW FROM starttime) in (0,6)
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

So the average trip duration time during weekends is 662.94 s

### c. do the same for weekday

In [ ]:
test = '''
SELECT * FROM citibike
WHERE extract(DOW FROM starttime) not in (0,6)
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

In [ ]:
test = '''
SELECT AVG(tripduration) as avgtrips FROM citibike
WHERE extract(DOW FROM starttime) not in (0,6)
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

The average trip duration for weekday is 662.94 s

## Task 3
### a. Showing the list of start station locations

In [ ]:
test = '''
SELECT CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) 
             as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id,
       MIN(start_station_name) as start_station_name 
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

### b. Showing the numbers of trips started per station but only for stations within 500m of Time Square

In [ ]:
test = '''
SELECT CDB_TransformToWebmercator(
                CDB_LatLng(start_station_latitude, 
                           start_station_longitude)
       ) as the_geom_webmercator, 
       MIN(cartodb_id) as cartodb_id,
       COUNT(tripduration) as numtrips,
       MIN(start_station_name) as start_station_name
FROM citibike
WHERE ST_DWithin(
        CDB_LatLng(start_station_latitude,
                   start_station_longitude)::geography,
        CDB_LatLng(40.7577, -73.9857)::geography,
      500)
GROUP BY start_station_latitude, start_station_longitude
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

## Task 4   
## Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!

In [ ]:
test = '''
SELECT CDB_TransformToWebmercator(
                CDB_LatLng(start_station_latitude, 
                           start_station_longitude)
       ) as the_geom_webmercator, 
       MIN(cartodb_id) as cartodb_id,
       COUNT(tripduration) as numtrips,
       AVG(tripduration) as avgtrips,
       MIN(start_station_name) as start_station_name
FROM citibike
WHERE ST_DWithin(
        CDB_LatLng(start_station_latitude,
                   start_station_longitude)::geography,
        CDB_LatLng(40.7577, -73.9857)::geography,
      500) and extract(DOW FROM starttime) in (0,6)
GROUP BY start_station_latitude, start_station_longitude
ORDER BY avgtrips DESC
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

So the station that had the longest average trip duration during weekends and within 500m of TimeSquare is Broadway & W 49 St.

## Extra for SQL
##     Create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. 
## The number of trips per each pair of stations are output as attributes of these lines.

In [ ]:
test = '''
SELECT ST_MakeLine(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude,start_station_longitude)),
             CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude,end_station_longitude)))
                   as the_geom_webmercator,
     MIN(cartodb_id) as cartodb_id,
     COUNT(tripduration) as numtrips,
     MIN(start_station_name) as start_station_name,
     MIN(end_station_name) as end_station_name
FROM citibike
WHERE ST_DWithin(
      CDB_LatLng(start_station_latitude,
            start_station_longitude)::geography,
      CDB_LatLng(40.7577,-73.9857)::geography,
    500)
    AND tripduration<7200
GROUP BY start_station_latitude, start_station_longitude, end_station_latitude, end_station_longitude
'''

In [ ]:
pd.read_csv(io.BytesIO(queryCartoDB(test)), sep=',')

# extra credit: make the function python 2 and 3 compatible so that it works on the  PUI2016_Python3 kernel

##### I've made the function python 2 and 3 compatible.

## ASH - SCORE 100, Extra CRedit 0